In [2]:
import time 

import numpy as np
from numba import njit
from matplotlib import pyplot as plt
import sys
sys.path.append("../")
from difference_matrix import Difference_Matrix

### Styles of Recursion

Tail and Head Recursion

In [2]:
@njit(nogil=True,fastmath=True,cache=True)
def sherman_morrison_recursion(a_ij,DDT_inv):
    """Compute the inverse of a matrix using the Sherman-Morrison formula
    
        Utilizes forward recrusion for computational efficiency
    """
    
    A_inv=DDT_inv
    k=1
    
    # Loop over the columns of the matrix
    while k<=len(a_ij):

        # Create the vectors u and v which are scaled unit vectors
        e_n=np.zeros(len(a_ij))
        e_n[k-1]=1
        u=e_n.reshape(-1,1)
        v=e_n.reshape(1,-1)
       
        num=np.dot(np.dot(A_inv,np.dot(a_ij[k-1]*u,v)),A_inv)
        den=1+np.dot(np.dot(a_ij[k-1]*v,A_inv),u)
     
        A_inv=A_inv -num/den
   
        k=k+1
  

    return A_inv

In [3]:
e_n=np.zeros(10)
e_n.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

In [4]:
def test_sherman_morrison(n):
    diff=Difference_Matrix(n,2)

    DDT=diff.DDT
    DDT_inv=diff.DDT_inv

    a_ij=np.random.rand(DDT.shape[0])

    # compute the inverse of DDT+a_ij*I using the Sherman-Morrison formula
    start_time=time.time()
    A_inv=sherman_morrison_recursion(a_ij,DDT_inv)
    total_time=time.time()-start_time
    

    # check that the inverse is correct
    computed_inv=np.dot(A_inv,DDT+np.diag(a_ij))

    print(f"Max error: {np.max(np.abs(computed_inv-np.eye(DDT.shape[0])))}")
    print(f"Total time: {total_time}")

    return
    




### 10 by 10 System

In [5]:
test_sherman_morrison(10)

Max error: 4.0158848468863084e-15
Total time: 2.486875534057617


### 100 by 100 System

In [6]:
test_sherman_morrison(100)

Max error: 3.017628122179604e-11
Total time: 0.009695291519165039


### 1000 x 1000 system; poor scaling

In [7]:
test_sherman_morrison(1000)

Max error: 8.966299447593245e-08
Total time: 62.68518400192261


### Improvement via Woodburry Matrix Identity (Blocked Sherman Morrison)

In [16]:

def woodbury_matrix_inversion(a_ij,DDT_inv,step=20):
    """Compute the inverse of a matrix using the Woodbury formula
    
       k-blocks of the matrix are inverted at a time
    """
   
    A_inv=DDT_inv
    k=1
    
    # Loop over the columns of the matrix
    while k<=len(a_ij):


        len_block=min(len(a_ij[k-1:k-1+step]),step)

        # Create the vectors u and v which are scaled unit vectors
        u=np.zeros((len(a_ij),len_block))
        v=np.zeros((len_block,len(a_ij)))
        c=np.zeros((len_block,len_block))

     

        for i in range(0,len_block):
         
            u[k+i-1,i]=1
            v[i,k+i-1]=1
            c[i,i]=a_ij[k+i-1]
        
        #  extract kth block of A_inv
        truncated_mat=v.dot(A_inv).dot(u)

        # compute the inverse of the kth block of A_inv
        inv_truncated_mat=np.linalg.inv(truncated_mat)

        # check that the inverse using numpy is correct
        #assert np.max(np.abs(truncated_mat.dot(inv_truncated_mat)-np.eye(len_block)))<1e-10
        
        c_a_inv=sherman_morrison_recursion(1/a_ij[k-1:k-1+step],inv_truncated_mat)

   
     
        A_inv=A_inv -A_inv.dot(u).dot(c_a_inv.dot(v).dot(A_inv))
   
        k=k+step
    return A_inv

In [17]:
def test_woodbury_inversion(n,step=None):

    diff=Difference_Matrix(n,2)

    DDT=diff.DDT
    DDT_inv=diff.DDT_inv

    a_ij=np.random.rand(DDT.shape[0])

    start_time=time.time()
    if step is None:
        # compute the inverse of DDT+a_ij*I using the Woodbury formula
        A_inv=woodbury_matrix_inversion(a_ij,DDT_inv)
    else:
        # compute the inverse of DDT+a_ij*I using the Woodbury formula
        A_inv=woodbury_matrix_inversion(a_ij,DDT_inv,step=step)
    total_time=time.time()-start_time

    # check that the inverse is correct
    computed_inv=A_inv.dot(DDT+np.diag(a_ij))

    print(f"Max error: {np.max(np.abs(computed_inv-np.eye(DDT.shape[0])))}")
    print(f"Total time: {total_time}")

    return

In [24]:
test_woodbury_inversion(500,step=20)

Max error: 3.534012004535493e-08
Total time: 0.07909440994262695


In [12]:
test_woodbury_inversion(1000)

Max error: 8.123289785843627e-08
Total time: 0.36255598068237305


In [66]:
test_woodbury_inversion(2500)


Max error: 1.5419815433619612e-06
Total time: 15.629467010498047


In [44]:
test_woodbury_inversion(5000)

Max error: 2.6600141148404054e-05
Total time: 123.6377604007721
